In [9]:
import pandas as pd
import torch
import sys
import numpy as np
from tqdm import tqdm
from numpy.linalg import eigvals, matrix_rank
sys.path.append("../utility")
from network import KoopmanNet

project_name = "May_7_control_loss"

In [10]:
log = pd.read_csv(f"../log/{project_name}/koopman_results_log.csv")
log

,env_name,state_dim,u_dim,train_samples,val_samples,test_samples,Ksteps,train_steps,batch_size,initial_lr,...,seed,normalize,best_val_Kloss,convergence_val_Kloss,test_Kloss,test_CovLoss,test_ControlLoss,model_path,num_params,encoder_num_params
0,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.006289,0.006294,0.006309,0.904466,0.996261,../log/May_7_control_loss/best_models/20250507...,135218,135176
1,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.006203,0.006208,0.006214,11.427120,2.545041,../log/May_7_control_loss/best_models/20250507...,135218,135176
2,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005938,0.005942,0.005973,278.827148,0.996187,../log/May_7_control_loss/best_models/20250507...,141686,141344
3,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005685,0.005689,0.005714,2021.217285,2.218377,../log/May_7_control_loss/best_models/20250507...,141686,141344
4,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005704,0.005713,0.005742,2888.298096,0.996035,../log/May_7_control_loss/best_models/20250507...,170438,166016
5,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005654,0.005663,0.005691,18542.750000,1.206878,../log/May_7_control_loss/best_models/20250507...,170438,166016
6,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005682,0.005689,0.005711,2322.650879,0.995482,../log/May_7_control_loss/best_models/20250507...,265734,198912
7,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005657,0.005664,0.005690,59155.789062,1.530898,../log/May_7_control_loss/best_models/20250507...,265734,198912
8,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005666,0.005674,0.005703,5663.848633,0.995542,../log/May_7_control_loss/best_models/20250507...,1713158,659456
9,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005645,0.005652,0.005687,227897.984375,1.298934,../log/May_7_control_loss/best_models/20250507...,1713158,659456


In [19]:
df_new = log[(log['encode_dim'] != 1024)]
df_new

,env_name,state_dim,u_dim,train_samples,val_samples,test_samples,Ksteps,train_steps,batch_size,initial_lr,...,seed,normalize,best_val_Kloss,convergence_val_Kloss,test_Kloss,test_CovLoss,test_ControlLoss,model_path,num_params,encoder_num_params
0,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.006289,0.006294,0.006309,0.904466,0.996261,../log/May_7_control_loss/best_models/20250507...,135218,135176
1,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.006203,0.006208,0.006214,11.427120,2.545041,../log/May_7_control_loss/best_models/20250507...,135218,135176
2,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005938,0.005942,0.005973,278.827148,0.996187,../log/May_7_control_loss/best_models/20250507...,141686,141344
3,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005685,0.005689,0.005714,2021.217285,2.218377,../log/May_7_control_loss/best_models/20250507...,141686,141344
4,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005704,0.005713,0.005742,2888.298096,0.996035,../log/May_7_control_loss/best_models/20250507...,170438,166016
5,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005654,0.005663,0.005691,18542.750000,1.206878,../log/May_7_control_loss/best_models/20250507...,170438,166016
6,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005682,0.005689,0.005711,2322.650879,0.995482,../log/May_7_control_loss/best_models/20250507...,265734,198912
7,DampingPendulum,2,1,60000,20000,20000,15,60000,64,0.001,...,1,True,0.005657,0.005664,0.005690,59155.789062,1.530898,../log/May_7_control_loss/best_models/20250507...,265734,198912
10,DoublePendulum,4,2,60000,20000,20000,15,60000,64,0.001,...,1,True,0.192329,0.192348,0.193131,1.681498,0.882910,../log/May_7_control_loss/best_models/20250507...,136280,136200
11,DoublePendulum,4,2,60000,20000,20000,15,60000,64,0.001,...,1,True,0.180227,0.180268,0.181543,1.956149,2829.913330,../log/May_7_control_loss/best_models/20250507...,136280,136200


In [20]:
def calculate_controllability(df, ckpt_path):
    ckpt = torch.load(ckpt_path)
    layers = ckpt.get("encode_layers", ckpt["layer"])
    feature_dim = layers[-1]
    row = df.loc[df['model_path'] == ckpt_path]
    state_dim = row.iloc[0].state_dim
    u_dim = row.iloc[0].u_dim
    Nkoop = state_dim + df_new.iloc[idx].encode_dim
    control_dim = u_dim
    use_residual = row.iloc[0].use_residual
    net = KoopmanNet(layers, Nkoop, control_dim, use_residual=use_residual)
    net.load_state_dict(ckpt.get("state_dict", ckpt["model"]))
    net.eval()
    
    A = net.lA.weight.detach().numpy()
    B = net.lB.weight.detach().numpy()
    n = A.shape[0]
    C_n = B
    for i in range(1, n):
        C_n = np.hstack((C_n, np.linalg.matrix_power(A, i) @ B))
    C_square = C_n @ C_n.T
    eig_C = eigvals(C_square)

    return np.max(np.abs(eig_C)), np.min(np.abs(eig_C))

In [21]:
results = []
for idx in tqdm(range(len(df_new))):
    model_path = df_new.iloc[idx].model_path
    max_eig, min_eig = calculate_controllability(df_new, model_path)
    results.append({
        "model_path": model_path,
        "max_eigenvalue": max_eig,
        "min_eigenvalue": min_eig
    })
results_df = pd.DataFrame(results)
results_df

100%|██████████| 16/16 [00:25<00:00,  1.60s/it]


,model_path,max_eigenvalue,min_eigenvalue
0,../log/May_7_control_loss/best_models/20250507...,2.525437,2.531368e-08
1,../log/May_7_control_loss/best_models/20250507...,0.547593,2.642434e-09
2,../log/May_7_control_loss/best_models/20250507...,57.267555,3.736520e-08
3,../log/May_7_control_loss/best_models/20250507...,5.911482,1.335367e-08
4,../log/May_7_control_loss/best_models/20250507...,71.531212,1.636506e-09
5,../log/May_7_control_loss/best_models/20250507...,21.812645,3.416520e-09
6,../log/May_7_control_loss/best_models/20250507...,198.245148,4.109929e-09
7,../log/May_7_control_loss/best_models/20250507...,116.289116,3.362076e-09
8,../log/May_7_control_loss/best_models/20250507...,1.220376,4.978524e-06
9,../log/May_7_control_loss/best_models/20250507...,1.727200,3.852160e-05


In [23]:
df_new[["env_name", "encode_dim", "use_control_loss", "model_path"]].merge(results_df, on="model_path").drop(columns=["model_path"])

,env_name,encode_dim,use_control_loss,max_eigenvalue,min_eigenvalue
0,DampingPendulum,4,True,2.525437,2.531368e-08
1,DampingPendulum,4,False,0.547593,2.642434e-09
2,DampingPendulum,16,True,57.267555,3.736520e-08
3,DampingPendulum,16,False,5.911482,1.335367e-08
4,DampingPendulum,64,True,71.531212,1.636506e-09
5,DampingPendulum,64,False,21.812645,3.416520e-09
6,DampingPendulum,256,True,198.245148,4.109929e-09
7,DampingPendulum,256,False,116.289116,3.362076e-09
8,DoublePendulum,4,True,1.220376,4.978524e-06
9,DoublePendulum,4,False,1.727200,3.852160e-05
